# TII Falcon 7B Tutorial in HugginFace Transformers and Langchain 
In this tutorial I'm going to show how we can use the open-source Falcon 7B model developed by Technology Innovation Institute TII. Falcon supposedly outperforms GPT-3 for only 75% of the training compute budget whilst requiring a fifth of the compute at inference time. 


1. Installing packages

In [1]:
!pip install -q transformers 
!pip install -q accelerate einops 
!pip install -q langchain 
!pip install -q xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.4/969.4 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Let's initialize a HuggingFace pipeline and load the LLM

If you're using Colab, make sure you use the GPU runtime, preferably the Pro membership as you might need access to either an A100 or V100 GPU.  
Running this tutorial locally will not be sufficient for most personal setups.  
The general steps that will be taken:  
1. Loading the Falcon model from HuggingFace 
2. Loading the model tokenizer 

In [ ]:
from torch import cuda, bfloat16 
import transformers 
from transformers import AutoTokenizer, AutoModelForCausalLM 
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'  

#creating a model 
fmodel = AutoModelForCausalLM.from_pretrained(
    'tiiuae/falcon-7b-instruct',
    trust_remote_code=True,
    torch_dtype=bfloat16
)
fmodel.eval() 
fmodel.to(device) 
print(f'Model loaded on {device}') 

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


The tokenizer is loaded from the tii model itself, as implemented below. 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('tiiuae/falcon-7b-instruct')

Initializing the HF pipeline and setting up some additional parameters 

In [ ]:
gen_text = transformers.pipeline(
    model=fmodel, 
    tokenizer=tokenizer, 
    task='text-generation', 
    return_full_text=True, 
    device=device, 
    max_length=10000, 
    temperature=0.1, 
    top_p=0.15, #select from top tokens whose probability adds up to 15%
    top_k=0, #selecting from top 0 tokens 
    repetition_penalty=1.1, #without a penalty, output starts to repeat 
    do_sample=True, 
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
result = gen_text("What is the name of the first president of the united arab emirates?") 
print(result[0]['generated_text']) 

### LangChain Implementation 

In [ ]:
from langchain import PromptTemplate, LLMChain 
from langchain.chains.conversation.memory import ConversationBufferMemory,ConversationSummaryMemory
from langchain.llms import HuggingFacePipeline 
#Creating a basic prompt template 

template = """
I want you to act as an informative AI assistant. 
Answer the following question: {question}.
"""
prompt = PromptTemplate(
    input_variables=['question'], 
    template=template 
)

In [ ]:
#showcasing the format of the prompt 
prompt.format(question="What is the name of the first US president? ")

In [ ]:
llm = HuggingFacePipeline(pipeline=gen_text)
llm_chain = LLMChain(llm=llm,prompt=prompt)

In [ ]:
llm_chain.predict(question='List me some of the most important electrolytes') 

### Implementing LangChain conversational memory

In [ ]:
#creating a new conversational template 
template = """You are an informative assistant chatting with a human. 
{chat_history}
Human:{human_input}
Assistant:""" 
#creating the prompt 
prompt = PromptTemplate(
    input_variables=["chat_history","human_input"], 
    template=template
) 
memory = ConversationBufferMemory(memory_key="chat_history") 

In [ ]:
llm = HuggingFacePipeline(pipeline=gen_text) 
#adding memory to the llm chain 
llm_chain = LLMChain(llm=llm,prompt=prompt,memory=memory) 

In [ ]:
#1st prompt 
llm_chain.predict(human_input='What is the first president of UAE?')

In [ ]:
#2nd prompt 
llm_chain.predict(human_input='Who was the president that came after him?')